In [1]:
!pip install opendatasets textstat

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.0 MB/s eta 0:00:00


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import opendatasets as od
import textstat

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

pd.set_option('display.max_columns', 100)

/kaggle/input/scrabble-player-rating/sample_submission.csv
/kaggle/input/scrabble-player-rating/games.csv
/kaggle/input/scrabble-player-rating/turns.csv
/kaggle/input/scrabble-player-rating/train.csv
/kaggle/input/scrabble-player-rating/test.csv


In [3]:
datadir = "/kaggle/input/scrabble-player-rating"

In [4]:
df_games = pd.read_csv(datadir+"/games.csv")
df_turns = pd.read_csv(datadir+"/turns.csv")
df_test = pd.read_csv(datadir+"/test.csv")
df_train = pd.read_csv(datadir+"/train.csv")
df_sample_submission = pd.read_csv(datadir+"/sample_submission.csv")

# Preprocessing Data

In [5]:
df_games.head(10)

,game_id,first,time_control_name,game_end_reason,winner,created_at,lexicon,initial_time_seconds,increment_seconds,rating_mode,max_overtime_minutes,game_duration_seconds
0,1,BetterBot,regular,STANDARD,1,2022-08-26 03:38:49,NWL20,1200,0,CASUAL,1,674.844274
1,2,Super,regular,STANDARD,1,2022-08-10 19:19:59,CSW21,3600,0,RATED,1,364.214418
2,3,BetterBot,regular,STANDARD,1,2022-09-04 08:04:27,CSW21,900,0,RATED,5,492.268262
3,4,BetterBot,regular,RESIGNED,0,2022-09-12 02:36:19,CSW21,3600,0,CASUAL,1,350.861141
4,5,STEEBot,regular,STANDARD,0,2022-09-06 04:31:36,NWL20,1200,0,CASUAL,1,642.688722
5,6,HivinD,regular,STANDARD,0,2022-08-21 14:56:35,CSW21,900,0,RATED,1,426.950541
6,7,Priya1,regular,STANDARD,1,2022-08-26 03:07:48,CSW21,1260,0,RATED,1,385.599607
7,8,HastyBot,regular,STANDARD,0,2022-09-06 21:03:24,CSW21,900,0,RATED,1,458.019893
8,9,BetterBot,regular,STANDARD,0,2022-09-02 01:55:00,NWL20,1200,0,RATED,1,567.144135
9,10,squashy,regular,STANDARD,0,2022-08-07 05:03:03,CSW21,900,0,RATED,1,118.911215


In [6]:
df_turns.head(10)

,game_id,turn_number,nickname,rack,location,move,points,score,turn_type
0,1,1,BetterBot,DDEGITT,8G,DIG,10,10,Play
1,1,2,stevy,AEHOPUX,7H,HAP,18,18,Play
2,1,3,BetterBot,DEELTTU,6I,LUTE,16,26,Play
3,1,4,stevy,EMORSUX,5K,UM,16,34,Play
4,1,5,BetterBot,ACDEITU,L5,..DICATE,28,54,Play
5,1,6,stevy,EEOORSX,M11,OXO,31,65,Play
6,1,7,BetterBot,AAEJRSU,5E,RAJAS,29,83,Play
7,1,8,stevy,AEEFIRS,9B,FAERIES,72,137,Play
8,1,9,BetterBot,ELNNORU,C3,NEURON.L,68,151,Play
9,1,10,stevy,EEEHIKO,6F,HO,35,172,Play


In [7]:
def fe_turns(df):
    df["rack_len"] = df["rack"].str.len()
    df["move_len"] = df["move"].str.len()
    df["move"].fillna("None",inplace=True)
    df["difficult_word"] = df["move"].apply(textstat.difficult_words)
    
    df["rack_len_less_than_7"] = df["rack_len"].apply(lambda x : x <7)
    rare_letters = ["Z", "Q", "J", "X", "K", "V", "Y", "W", "G"]
    df["difficult_letters"] = df["move"].apply(lambda x: len([letter for letter in x if letter in rare_letters]))
    df["points_per_letter"] = df["points"]/df["move_len"]
    df["direction_of_play"] = df["location"].apply(lambda x: 1 if str(x)[0].isdigit() else 0)
    df["curr_board_pieces_used"] = df["move"].apply(lambda x: str(x).count(".") + sum(int(c.islower()) for c in str(x)))
    
    df["turn_type"].fillna("None",inplace=True)
    turn_type_unique = df["turn_type"].unique()
    df = pd.get_dummies(df, columns=["turn_type"])
    dummy_features = [f"turn_type_{value}" for value in turn_type_unique]
    
    char_map = {
        'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7, 'H': 8,
        'I': 9, 'J': 10, 'K': 11, 'L': 12, 'M': 13, 'N': 14, 'O': 15,
    }
    df["loc_nan"] = df["location"].isna()
    df['y'] = df["location"].str.extract('(\d+)')[0].values
    df['y'].fillna("8",inplace=True)
    df["y"] = df["y"].astype(int) - 8
    
    df["x"] = df["location"].str.extract('([A-Z])')[0].values
    df["x"].replace(char_map, inplace=True)
    df['x'].fillna("8",inplace=True)
    df["x"] = df["x"].astype(int) - 8
    
    return df, dummy_features

In [8]:
turns_fe, dummy_features = fe_turns(df_turns.copy())
print(dummy_features)

['turn_type_Play', 'turn_type_End', 'turn_type_Exchange', 'turn_type_Pass', 'turn_type_Timeout', 'turn_type_Challenge', 'turn_type_Six-Zero Rule', 'turn_type_None']


In [9]:
turns_fe.head(10)

,game_id,turn_number,nickname,rack,location,move,points,score,rack_len,move_len,difficult_word,rack_len_less_than_7,difficult_letters,points_per_letter,direction_of_play,curr_board_pieces_used,turn_type_Challenge,turn_type_End,turn_type_Exchange,turn_type_None,turn_type_Pass,turn_type_Play,turn_type_Six-Zero Rule,turn_type_Timeout,loc_nan,y,x
0,1,1,BetterBot,DDEGITT,8G,DIG,10,10,7.0,3.0,0,False,1,3.333333,1,0,0,0,0,0,0,1,0,0,False,0,-1
1,1,2,stevy,AEHOPUX,7H,HAP,18,18,7.0,3.0,0,False,0,6.000000,1,0,0,0,0,0,0,1,0,0,False,-1,0
2,1,3,BetterBot,DEELTTU,6I,LUTE,16,26,7.0,4.0,0,False,0,4.000000,1,0,0,0,0,0,0,1,0,0,False,-2,1
3,1,4,stevy,EMORSUX,5K,UM,16,34,7.0,2.0,0,False,0,8.000000,1,0,0,0,0,0,0,1,0,0,False,-3,3
4,1,5,BetterBot,ACDEITU,L5,..DICATE,28,54,7.0,8.0,1,False,0,3.500000,0,2,0,0,0,0,0,1,0,0,False,-3,4
5,1,6,stevy,EEOORSX,M11,OXO,31,65,7.0,3.0,0,False,1,10.333333,0,0,0,0,0,0,0,1,0,0,False,3,5
6,1,7,BetterBot,AAEJRSU,5E,RAJAS,29,83,7.0,5.0,1,False,1,5.800000,1,0,0,0,0,0,0,1,0,0,False,-3,-3
7,1,8,stevy,AEEFIRS,9B,FAERIES,72,137,7.0,7.0,0,False,0,10.285714,1,0,0,0,0,0,0,1,0,0,False,1,-6
8,1,9,BetterBot,ELNNORU,C3,NEURON.L,68,151,7.0,8.0,1,False,0,8.500000,0,1,0,0,0,0,0,1,0,0,False,-5,-5
9,1,10,stevy,EEEHIKO,6F,HO,35,172,7.0,2.0,0,False,0,17.500000,1,0,0,0,0,0,0,1,0,0,False,-2,-2


In [10]:
turns_fe["turn_count"] = turns_fe["game_id"]

agg_func = {'points': ['mean'],
 'score': ['mean'],
 'rack_len': ['mean'],
 'move_len': ['mean', 'max'],
 'difficult_word': ['mean'],
 'loc_nan': ['mean'],
 'x': ['mean'],
 'y': ['mean'],
 'rack_len_less_than_7': ['mean'],
 'difficult_letters': ['mean'],
 'points_per_letter': ['mean'],
 'direction_of_play': ['mean'],
 'curr_board_pieces_used': ['mean'],
 'turn_type_Play': ['mean'],
 'turn_type_End': ['mean'],
 'turn_type_Exchange': ['mean'],
 'turn_type_Pass': ['mean'],
 'turn_type_Timeout': ['mean'],
 'turn_type_Challenge': ['mean'],
 'turn_type_Six-Zero Rule': ['mean'],
 'turn_type_None': ['mean'],
 'turn_count': ['count']}


In [11]:
turns_group = turns_fe.groupby(["game_id", "nickname"], as_index=False).agg(agg_func)
turns_group.head(10)

game_id        nickname     points       score  rack_len  move_len       \
                                mean        mean      mean      mean  max   
0       1       BetterBot  25.769231  182.230769  6.769231  4.615385  8.0   
1       1           stevy  30.642857  256.500000  6.538462  3.857143  7.0   
2       2       BetterBot  33.416667  229.250000  6.916667  5.250000  8.0   
3       2           Super  34.857143  282.285714  7.000000  4.357143  8.0   
4       3       BetterBot  24.461538  155.076923  7.000000  4.384615  8.0   
5       3       davidavid  31.428571  231.714286  6.692308  4.357143  7.0   
6       4       BetterBot  31.866667  228.666667  7.000000  5.066667  8.0   
7       4  Inandoutworker   8.500000   85.000000  7.000000  2.928571  7.0   
8       5         STEEBot  28.466667  246.266667  6.800000  4.466667  9.0   
9       5           stevy  20.312500  203.312500  6.466667  3.375000  6.0   

  difficult_word   loc_nan         x         y rack_len_less_than_7  \
            mean      mean      mean      mean                 mean   
0       0.230769  0.000000  0.538462 -2.769231             0.076923   
1       0.071429  0.071429 -0.500000 -0.071429             0.142857   
2       0.250000  0.000000 -1.750000 -1.416667             0.083333   
3       0.357143  0.214286 -0.357143  0.500000             0.000000   
4       0.153846  0.076923 -1.384615 -0.230769             0.000000   
5       0.142857  0.071429  0.285714 -0.142857             0.071429   
6       0.200000  0.000000 -1.600000 -2.933333             0.000000   
7       0.071429  0.142857 -0.357143 -1.214286             0.000000   
8       0.133333  0.000000  0.466667  0.466667             0.066667   
9       0.062500  0.125000  0.937500 -0.875000             0.125000   

  difficult_letters points_per_letter direction_of_play  \
               mean              mean              mean   
0          0.846154          6.042125          0.384615   
1          0.214286          7.773980          0.714286   
2          0.583333          5.967460          0.500000   
3          0.571429          8.023724          0.428571   
4          0.615385          5.645421          0.538462   
5          0.571429          7.911735          0.142857   
6          0.333333          6.456429          0.400000   
7          0.571429          2.898810          0.642857   
8          0.333333          6.131799          0.666667   
9          0.562500          7.338542          0.500000   

  curr_board_pieces_used turn_type_Play turn_type_End turn_type_Exchange  \
                    mean           mean          mean               mean   
0               0.615385       1.000000      0.000000           0.000000   
1               0.428571       0.928571      0.071429           0.000000   
2               1.083333       1.000000      0.000000           0.000000   
3               0.285714       0.785714      0.071429           0.071429   
4               0.692308       0.923077      0.000000           0.076923   
5               0.428571       0.928571      0.071429           0.000000   
6               1.400000       1.000000      0.000000           0.000000   
7               0.928571       0.857143      0.000000           0.142857   
8               0.733333       1.000000      0.000000           0.000000   
9               0.312500       0.875000      0.062500           0.062500   

  turn_type_Pass turn_type_Timeout turn_type_Challenge  \
            mean              mean                mean   
0       0.000000               0.0                 0.0   
1       0.000000               0.0                 0.0   
2       0.000000               0.0                 0.0   
3       0.071429               0.0                 0.0   
4       0.000000               0.0                 0.0   
5       0.000000               0.0                 0.0   
6       0.000000               0.0                 0.0   
7       0.000000               0.0                 0.0   
8       0.000000      

In [12]:
def feature_agg(df, turns_group, games):
    df["games_played"] = df["nickname"].replace(df["nickname"].value_counts())
    
    df = df.merge(turns_group, left_on=["game_id", "nickname"], right_on=["game_id", "nickname"], how='left')
    
    
    games["created_at"] = pd.to_datetime(games["created_at"])
    df = df.merge(games, left_on="game_id", right_on='game_id', how='left')
    
    
    df["first"]=  df["first"] == df["nickname"]
    df["first"] = df["first"].replace({True: 1, False: 0})
    return df

train_fe = feature_agg(df_train, turns_group, df_games)
test_fe = feature_agg(df_test, turns_group, df_games)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:9203: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left,2 on the right)
  validate=validate,
/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [13]:
test_fe[test_fe['rating'].notnull()]['nickname'].unique()

array(['BetterBot', 'STEEBot', 'HastyBot'], dtype=object)

In [14]:
train_fe.head(10)

,game_id,nickname,score,rating,games_played,"(points, mean)","(score, mean)","(rack_len, mean)","(move_len, mean)","(move_len, max)","(difficult_word, mean)","(loc_nan, mean)","(x, mean)","(y, mean)","(rack_len_less_than_7, mean)","(difficult_letters, mean)","(points_per_letter, mean)","(direction_of_play, mean)","(curr_board_pieces_used, mean)","(turn_type_Play, mean)","(turn_type_End, mean)","(turn_type_Exchange, mean)","(turn_type_Pass, mean)","(turn_type_Timeout, mean)","(turn_type_Challenge, mean)","(turn_type_Six-Zero Rule, mean)","(turn_type_None, mean)","(turn_count, count)",first,time_control_name,game_end_reason,winner,created_at,lexicon,initial_time_seconds,increment_seconds,rating_mode,max_overtime_minutes,game_duration_seconds
0,1,BetterBot,335,1637,16602,25.769231,182.230769,6.769231,4.615385,8.0,0.230769,0.000000,0.538462,-2.769231,0.076923,0.846154,6.042125,0.384615,0.615385,1.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,13,1,regular,STANDARD,1,2022-08-26 03:38:49,NWL20,1200,0,CASUAL,1,674.844274
1,1,stevy,429,1500,410,30.642857,256.500000,6.538462,3.857143,7.0,0.071429,0.071429,-0.500000,-0.071429,0.142857,0.214286,7.773980,0.714286,0.428571,0.928571,0.071429,0.000000,0.0,0.0,0.0,0.0,0.0,14,0,regular,STANDARD,1,2022-08-26 03:38:49,NWL20,1200,0,CASUAL,1,674.844274
2,3,davidavid,440,1811,254,31.428571,231.714286,6.692308,4.357143,7.0,0.142857,0.071429,0.285714,-0.142857,0.071429,0.571429,7.911735,0.142857,0.428571,0.928571,0.071429,0.000000,0.0,0.0,0.0,0.0,0.0,14,0,regular,STANDARD,1,2022-09-04 08:04:27,CSW21,900,0,RATED,5,492.268262
3,3,BetterBot,318,2071,16602,24.461538,155.076923,7.000000,4.384615,8.0,0.153846,0.076923,-1.384615,-0.230769,0.000000,0.615385,5.645421,0.538462,0.692308,0.923077,0.000000,0.076923,0.0,0.0,0.0,0.0,0.0,13,1,regular,STANDARD,1,2022-09-04 08:04:27,CSW21,900,0,RATED,5,492.268262
4,4,Inandoutworker,119,1473,205,8.500000,85.000000,7.000000,2.928571,7.0,0.071429,0.142857,-0.357143,-1.214286,0.000000,0.571429,2.898810,0.642857,0.928571,0.857143,0.000000,0.142857,0.0,0.0,0.0,0.0,0.0,14,0,regular,RESIGNED,0,2022-09-12 02:36:19,CSW21,3600,0,CASUAL,1,350.861141
5,4,BetterBot,478,1936,16602,31.866667,228.666667,7.000000,5.066667,8.0,0.200000,0.000000,-1.600000,-2.933333,0.000000,0.333333,6.456429,0.400000,1.400000,1.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,15,1,regular,RESIGNED,0,2022-09-12 02:36:19,CSW21,3600,0,CASUAL,1,350.861141
6,5,stevy,325,1500,410,20.312500,203.312500,6.466667,3.375000,6.0,0.062500,0.125000,0.937500,-0.875000,0.125000,0.562500,7.338542,0.500000,0.312500,0.875000,0.062500,0.062500,0.0,0.0,0.0,0.0,0.0,16,0,regular,STANDARD,0,2022-09-06 04:31:36,NWL20,1200,0,CASUAL,1,642.688722
7,5,STEEBot,427,1844,16462,28.466667,246.266667,6.800000,4.466667,9.0,0.133333,0.000000,0.466667,0.466667,0.066667,0.333333,6.131799,0.666667,0.733333,1.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,15,1,regular,STANDARD,0,2022-09-06 04:31:36,NWL20,1200,0,CASUAL,1,642.688722
8,6,HivinD,378,2029,592,31.500000,184.333333,7.000000,4.750000,6.0,0.166667,0.000000,-2.083333,0.166667,0.000000,0.666667,6.718056,0.500000,0.666667,1.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,12,1,regular,STANDARD,0,2022-08-21 14:56:35,CSW21,900,0,RATED,1,426.950541
9,6,STEEBot,427,2143,16462,32.846154,228.384615,6.500000,5.076923,8.0,0.153846,0.153846,-1.769231,-2.000000,0.076923,0.461538,6.356502,0.307692,0.692308,0.846154,0.076923,0.076923,0.0,0.0,0.0,0.0,0.0,13,0,regular,STANDARD,0,2022-08-21 14:56:35,CSW21,900,0,RATED,1,426.950541


In [15]:
bot_names =["BetterBot", "STEEBot", "HastyBot"]
train_fe=train_fe[-train_fe['nickname'].isin(bot_names)]

In [16]:
from sklearn.preprocessing import LabelEncoder

def encode_cat(df):
    df = pd.get_dummies(df, columns=["time_control_name"])
    df = pd.get_dummies(df, columns=["game_end_reason"])
    df = pd.get_dummies(df, columns=["lexicon"])
    df = pd.get_dummies(df, columns=["rating_mode"])
    
    le = LabelEncoder()
    le.fit(df['nickname'])
    df['nickname'] = le.transform(df['nickname'])

    return df


train_set = encode_cat(train_fe)
test_set = encode_cat(test_fe)

train_set.isna().sum()

game_id                               0
nickname                              0
score                                 0
rating                                0
games_played                          0
(points, mean)                        0
(score, mean)                         0
(rack_len, mean)                      0
(move_len, mean)                      0
(move_len, max)                       0
(difficult_word, mean)                0
(loc_nan, mean)                       0
(x, mean)                             0
(y, mean)                             0
(rack_len_less_than_7, mean)          0
(difficult_letters, mean)             0
(points_per_letter, mean)             0
(direction_of_play, mean)             0
(curr_board_pieces_used, mean)        0
(turn_type_Play, mean)                0
(turn_type_End, mean)                 0
(turn_type_Exchange, mean)            0
(turn_type_Pass, mean)                0
(turn_type_Timeout, mean)             0
(turn_type_Challenge, mean)           0


In [17]:
test_set['rating'].isnull().sum()

22363

In [18]:
test_set=test_set[test_set['rating'].isnull()]

In [19]:
test_set=test_set.drop(['rating','created_at'], axis=1)

In [20]:
test_set.insert(43,'lexicon_NSWL20',0)

In [21]:
train_set.head()

,game_id,nickname,score,rating,games_played,"(points, mean)","(score, mean)","(rack_len, mean)","(move_len, mean)","(move_len, max)","(difficult_word, mean)","(loc_nan, mean)","(x, mean)","(y, mean)","(rack_len_less_than_7, mean)","(difficult_letters, mean)","(points_per_letter, mean)","(direction_of_play, mean)","(curr_board_pieces_used, mean)","(turn_type_Play, mean)","(turn_type_End, mean)","(turn_type_Exchange, mean)","(turn_type_Pass, mean)","(turn_type_Timeout, mean)","(turn_type_Challenge, mean)","(turn_type_Six-Zero Rule, mean)","(turn_type_None, mean)","(turn_count, count)",first,winner,created_at,initial_time_seconds,increment_seconds,max_overtime_minutes,game_duration_seconds,time_control_name_blitz,time_control_name_rapid,time_control_name_regular,time_control_name_ultrablitz,game_end_reason_CONSECUTIVE_ZEROES,game_end_reason_RESIGNED,game_end_reason_STANDARD,game_end_reason_TIME,lexicon_CSW21,lexicon_ECWL,lexicon_NSWL20,lexicon_NWL20,rating_mode_CASUAL,rating_mode_RATED
1,1,957,429,1500,410,30.642857,256.500000,6.538462,3.857143,7.0,0.071429,0.071429,-0.500000,-0.071429,0.142857,0.214286,7.773980,0.714286,0.428571,0.928571,0.071429,0.000000,0.0,0.0,0.0,0.0,0.0,14,0,1,2022-08-26 03:38:49,1200,0,1,674.844274,0,0,1,0,0,0,1,0,0,0,0,1,1,0
2,3,708,440,1811,254,31.428571,231.714286,6.692308,4.357143,7.0,0.142857,0.071429,0.285714,-0.142857,0.071429,0.571429,7.911735,0.142857,0.428571,0.928571,0.071429,0.000000,0.0,0.0,0.0,0.0,0.0,14,0,1,2022-09-04 08:04:27,900,0,5,492.268262,0,0,1,0,0,0,1,0,1,0,0,0,0,1
4,4,224,119,1473,205,8.500000,85.000000,7.000000,2.928571,7.0,0.071429,0.142857,-0.357143,-1.214286,0.000000,0.571429,2.898810,0.642857,0.928571,0.857143,0.000000,0.142857,0.0,0.0,0.0,0.0,0.0,14,0,0,2022-09-12 02:36:19,3600,0,1,350.861141,0,0,1,0,0,1,0,0,1,0,0,0,1,0
6,5,957,325,1500,410,20.312500,203.312500,6.466667,3.375000,6.0,0.062500,0.125000,0.937500,-0.875000,0.125000,0.562500,7.338542,0.500000,0.312500,0.875000,0.062500,0.062500,0.0,0.0,0.0,0.0,0.0,16,0,0,2022-09-06 04:31:36,1200,0,1,642.688722,0,0,1,0,0,0,1,0,0,0,0,1,1,0
8,6,212,378,2029,592,31.500000,184.333333,7.000000,4.750000,6.0,0.166667,0.000000,-2.083333,0.166667,0.000000,0.666667,6.718056,0.500000,0.666667,1.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,12,1,0,2022-08-21 14:56:35,900,0,1,426.950541,0,0,1,0,0,0,1,0,1,0,0,0,0,1


In [22]:
test_set.head()

,game_id,nickname,score,games_played,"(points, mean)","(score, mean)","(rack_len, mean)","(move_len, mean)","(move_len, max)","(difficult_word, mean)","(loc_nan, mean)","(x, mean)","(y, mean)","(rack_len_less_than_7, mean)","(difficult_letters, mean)","(points_per_letter, mean)","(direction_of_play, mean)","(curr_board_pieces_used, mean)","(turn_type_Play, mean)","(turn_type_End, mean)","(turn_type_Exchange, mean)","(turn_type_Pass, mean)","(turn_type_Timeout, mean)","(turn_type_Challenge, mean)","(turn_type_Six-Zero Rule, mean)","(turn_type_None, mean)","(turn_count, count)",first,winner,initial_time_seconds,increment_seconds,max_overtime_minutes,game_duration_seconds,time_control_name_blitz,time_control_name_rapid,time_control_name_regular,time_control_name_ultrablitz,game_end_reason_CONSECUTIVE_ZEROES,game_end_reason_RESIGNED,game_end_reason_STANDARD,game_end_reason_TIME,lexicon_CSW21,lexicon_ECWL,lexicon_NSWL20,lexicon_NWL20,rating_mode_CASUAL,rating_mode_RATED
0,2,249,488,140,34.857143,282.285714,7.000000,4.357143,8.0,0.357143,0.214286,-0.357143,0.500000,0.000000,0.571429,8.023724,0.428571,0.285714,0.785714,0.071429,0.071429,0.071429,0.0,0.0,0.0,0.0,14,1,1,3600,0,1,364.214418,0,0,1,0,0,0,1,0,1,0,0,0,0,1
3,7,197,379,166,29.153846,185.230769,7.000000,3.846154,7.0,0.076923,0.076923,0.076923,1.230769,0.000000,0.538462,7.322894,0.615385,0.461538,0.923077,0.000000,0.076923,0.000000,0.0,0.0,0.0,0.0,13,1,1,1260,0,1,385.599607,0,0,1,0,0,0,1,0,1,0,0,0,0,1
5,11,259,462,269,33.000000,247.571429,6.857143,4.214286,8.0,0.214286,0.071429,-1.142857,-1.285714,0.071429,0.571429,8.737160,0.285714,0.285714,0.928571,0.000000,0.071429,0.000000,0.0,0.0,0.0,0.0,14,1,1,1200,0,1,501.739156,0,0,1,0,0,0,1,0,0,0,0,1,1,0
7,14,19,359,65,23.933333,244.800000,6.714286,4.066667,8.0,0.066667,0.066667,-0.733333,-1.200000,0.066667,0.400000,5.772222,0.333333,0.733333,0.933333,0.066667,0.000000,0.000000,0.0,0.0,0.0,0.0,15,0,0,900,0,1,293.253051,0,0,1,0,0,0,1,0,1,0,0,0,0,1
8,27,324,456,528,41.454545,269.272727,7.000000,4.454545,8.0,0.090909,0.000000,-1.181818,-3.000000,0.000000,0.545455,9.322727,0.454545,0.545455,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,11,1,1,1200,0,1,203.963230,0,0,1,0,0,0,1,0,1,0,0,0,0,1


# Model

In [23]:
X=train_set.drop(['rating','created_at'], axis=1)
y=train_set['rating'].values

In [24]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense , Dropout
from sklearn.metrics import mean_squared_error
n_cols = X.shape[1]


def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(50, input_shape=(n_cols,), activation= "relu"))
    model.add(Dense(50, activation= "relu"))
    model.add(Dense(50, activation= "relu"))
    model.add(Dense(50, activation= "relu"))
    model.add(Dense(1))
    model.compile(loss= ['mean_squared_error'] , optimizer=Adam(learning_rate=0.001), metrics=RootMeanSquaredError(name='rmse'))
    return model

model = baseline_model()
model.fit(X_train, y_train, epochs=50)

2022-12-15 18:52:00.525472: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2022-12-15 18:52:00.797385: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/50
1261/1261 [==============================] - 5s 4ms/step - loss: 136542.9219 - rmse: 369.5172
Epoch 2/50
1261/1261 [==============================] - 4s 3ms/step - loss: 72293.3594 - rmse: 268.8742
Epoch 3/50
1261/1261 [==============================] - 5s 4ms/step - loss: 64462.2969 - rmse: 253.8943
Epoch 4/50
1261/1261 [==============================] - 5s 4ms/step - loss: 58173.8594 - rmse: 241.1926
Epoch 5/50
1261/1261 [==============================] - 4s 3ms/step - loss: 52606.1133 - rmse: 229.3602
Epoch 6/50
1261/1261 [==============================] - 4s 4ms/step - loss: 45694.9102 - rmse: 213.7637
Epoch 7/50
1261/1261 [==============================] - 5s 4ms/step - loss: 42265.9961 - rmse: 205.5870
Epoch 8/50
1261/1261 [==============================] - 4s 3ms/step - loss: 36818.1211 - rmse: 191.8805
Epoch 9/50
1261/1261 [==============================] - 5s 4ms/step - loss: 33764.3086 - rmse: 183.7507
Epoch 10/50
1261/1261 [==============================] - 5s 4ms

In [26]:
y_train

array([1550, 2041, 1500, ..., 1470, 2013, 1754])

# Performance Evaluation

In [27]:
pred_train= model.predict(X_train)                                                                       
print(pred_train)
print("RMSE of train data: ",mean_squared_error(y_train,pred_train,squared=False))
pred= model.predict(X_test)
print("RMSE of test data: ",mean_squared_error(y_test,pred,squared=False))

[[1634.6024]
 [2019.2168]
 [1489.5509]
 ...
 [1554.0546]
 [1915.4636]
 [1928.6228]]
RMSE of train data:  132.0388180559818
RMSE of test data:  133.15818137435943


# Predecting

In [28]:
print(test_set.shape)

(22363, 47)


In [29]:
test_pred=model.predict(test_set)
test_set['rating']=(test_pred.round(decimals = 0)).astype(int)
test_set

,game_id,nickname,score,games_played,"(points, mean)","(score, mean)","(rack_len, mean)","(move_len, mean)","(move_len, max)","(difficult_word, mean)","(loc_nan, mean)","(x, mean)","(y, mean)","(rack_len_less_than_7, mean)","(difficult_letters, mean)","(points_per_letter, mean)","(direction_of_play, mean)","(curr_board_pieces_used, mean)","(turn_type_Play, mean)","(turn_type_End, mean)","(turn_type_Exchange, mean)","(turn_type_Pass, mean)","(turn_type_Timeout, mean)","(turn_type_Challenge, mean)","(turn_type_Six-Zero Rule, mean)","(turn_type_None, mean)","(turn_count, count)",first,winner,initial_time_seconds,increment_seconds,max_overtime_minutes,game_duration_seconds,time_control_name_blitz,time_control_name_rapid,time_control_name_regular,time_control_name_ultrablitz,game_end_reason_CONSECUTIVE_ZEROES,game_end_reason_RESIGNED,game_end_reason_STANDARD,game_end_reason_TIME,lexicon_CSW21,lexicon_ECWL,lexicon_NSWL20,lexicon_NWL20,rating_mode_CASUAL,rating_mode_RATED,rating
0,2,249,488,140,34.857143,282.285714,7.000000,4.357143,8.0,0.357143,0.214286,-0.357143,0.500000,0.000000,0.571429,8.023724,0.428571,0.285714,0.785714,0.071429,0.071429,0.071429,0.000000,0.0,0.0,0.0,14,1,1,3600,0,1,364.214418,0,0,1,0,0,0,1,0,1,0,0,0,0,1,1729
3,7,197,379,166,29.153846,185.230769,7.000000,3.846154,7.0,0.076923,0.076923,0.076923,1.230769,0.000000,0.538462,7.322894,0.615385,0.461538,0.923077,0.000000,0.076923,0.000000,0.000000,0.0,0.0,0.0,13,1,1,1260,0,1,385.599607,0,0,1,0,0,0,1,0,1,0,0,0,0,1,1809
5,11,259,462,269,33.000000,247.571429,6.857143,4.214286,8.0,0.214286,0.071429,-1.142857,-1.285714,0.071429,0.571429,8.737160,0.285714,0.285714,0.928571,0.000000,0.071429,0.000000,0.000000,0.0,0.0,0.0,14,1,1,1200,0,1,501.739156,0,0,1,0,0,0,1,0,0,0,0,1,1,0,1583
7,14,19,359,65,23.933333,244.800000,6.714286,4.066667,8.0,0.066667,0.066667,-0.733333,-1.200000,0.066667,0.400000,5.772222,0.333333,0.733333,0.933333,0.066667,0.000000,0.000000,0.000000,0.0,0.0,0.0,15,0,0,900,0,1,293.253051,0,0,1,0,0,0,1,0,1,0,0,0,0,1,2030
8,27,324,456,528,41.454545,269.272727,7.000000,4.454545,8.0,0.090909,0.000000,-1.181818,-3.000000,0.000000,0.545455,9.322727,0.454545,0.545455,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,11,1,1,1200,0,1,203.963230,0,0,1,0,0,0,1,0,1,0,0,0,0,1,2133
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44717,72760,422,494,316,49.400000,288.100000,7.000000,5.000000,8.0,0.400000,0.000000,-3.400000,-1.400000,0.000000,0.600000,9.817857,0.700000,0.500000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,10,0,1,1200,0,3,134.117320,0,0,1,0,0,0,1,0,1,0,0,0,0,1,2132
44718,72761,336,232,687,11.047619,156.047619,7.000000,3.285714,7.0,0.095238,0.285714,-1.000000,0.428571,0.000000,0.333333,1.653061,0.476190,0.238095,0.714286,0.000000,0.285714,0.000000,0.000000,0.0,0.0,0.0,21,1,0,1200,0,1,265.105576,0,0,1,0,0,0,1,0,1,0,0,0,0,1,2048
44721,72762,354,367,58,26.214286,199.357143,6.857143,3.642857,8.0,0.142857,0.071429,-2.000000,-0.071429,0.071429,0.357143,7.488690,0.571429,0.357143,0.928571,0.000000,0.071429,0.000000,0.000000,0.0,0.0,0.0,14,1,1,3600,0,1,386.086637,0,0,1,0,0,0,1,0,0,0,0,1,1,0,1724
44723,72768,152,357,56,29.750000,222.750000,6.636364,4.750000,7.0,0.083333,0.083333,-0.583333,-1.166667,0.083333,0.333333,5.908135,0.666667,0.583333,0.916667,0.083333,0.000000,0.000000,0.000000,0.0,0.0,0.0,12,0,0,3600,0,1,2155.922543,0,0,1,0,0,0,1,0,1,0,0,0,0,1,1774


In [30]:
train_set

,game_id,nickname,score,rating,games_played,"(points, mean)","(score, mean)","(rack_len, mean)","(move_len, mean)","(move_len, max)","(difficult_word, mean)","(loc_nan, mean)","(x, mean)","(y, mean)","(rack_len_less_than_7, mean)","(difficult_letters, mean)","(points_per_letter, mean)","(direction_of_play, mean)","(curr_board_pieces_used, mean)","(turn_type_Play, mean)","(turn_type_End, mean)","(turn_type_Exchange, mean)","(turn_type_Pass, mean)","(turn_type_Timeout, mean)","(turn_type_Challenge, mean)","(turn_type_Six-Zero Rule, mean)","(turn_type_None, mean)","(turn_count, count)",first,winner,created_at,initial_time_seconds,increment_seconds,max_overtime_minutes,game_duration_seconds,time_control_name_blitz,time_control_name_rapid,time_control_name_regular,time_control_name_ultrablitz,game_end_reason_CONSECUTIVE_ZEROES,game_end_reason_RESIGNED,game_end_reason_STANDARD,game_end_reason_TIME,lexicon_CSW21,lexicon_ECWL,lexicon_NSWL20,lexicon_NWL20,rating_mode_CASUAL,rating_mode_RATED
1,1,957,429,1500,410,30.642857,256.500000,6.538462,3.857143,7.0,0.071429,0.071429,-0.500000,-0.071429,0.142857,0.214286,7.773980,0.714286,0.428571,0.928571,0.071429,0.000000,0.0,0.0000,0.0,0.0,0.0,14,0,1,2022-08-26 03:38:49,1200,0,1,674.844274,0,0,1,0,0,0,1,0,0,0,0,1,1,0
2,3,708,440,1811,254,31.428571,231.714286,6.692308,4.357143,7.0,0.142857,0.071429,0.285714,-0.142857,0.071429,0.571429,7.911735,0.142857,0.428571,0.928571,0.071429,0.000000,0.0,0.0000,0.0,0.0,0.0,14,0,1,2022-09-04 08:04:27,900,0,5,492.268262,0,0,1,0,0,0,1,0,1,0,0,0,0,1
4,4,224,119,1473,205,8.500000,85.000000,7.000000,2.928571,7.0,0.071429,0.142857,-0.357143,-1.214286,0.000000,0.571429,2.898810,0.642857,0.928571,0.857143,0.000000,0.142857,0.0,0.0000,0.0,0.0,0.0,14,0,0,2022-09-12 02:36:19,3600,0,1,350.861141,0,0,1,0,0,1,0,0,1,0,0,0,1,0
6,5,957,325,1500,410,20.312500,203.312500,6.466667,3.375000,6.0,0.062500,0.125000,0.937500,-0.875000,0.125000,0.562500,7.338542,0.500000,0.312500,0.875000,0.062500,0.062500,0.0,0.0000,0.0,0.0,0.0,16,0,0,2022-09-06 04:31:36,1200,0,1,642.688722,0,0,1,0,0,0,1,0,0,0,0,1,1,0
8,6,212,378,2029,592,31.500000,184.333333,7.000000,4.750000,6.0,0.166667,0.000000,-2.083333,0.166667,0.000000,0.666667,6.718056,0.500000,0.666667,1.000000,0.000000,0.000000,0.0,0.0000,0.0,0.0,0.0,12,1,0,2022-08-21 14:56:35,900,0,1,426.950541,0,0,1,0,0,0,1,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100810,72767,190,340,1395,102,21.250000,179.062500,6.312500,3.500000,7.0,0.062500,0.062500,1.062500,0.250000,0.187500,0.375000,6.307887,0.437500,0.875000,0.937500,0.000000,0.000000,0.0,0.0625,0.0,0.0,0.0,16,0,0,2022-09-05 04:35:56,3600,0,1,3669.988413,0,0,1,0,0,0,0,1,1,0,0,0,1,0
100813,72770,921,97,1332,24,6.062500,62.625000,6.812500,3.187500,6.0,0.000000,0.062500,1.062500,0.687500,0.125000,0.062500,2.276042,0.562500,1.375000,0.937500,0.000000,0.000000,0.0,0.0625,0.0,0.0,0.0,16,0,0,2022-09-04 00:13:24,3600,0,1,3662.659287,0,0,1,0,0,0,0,1,1,0,0,0,0,1
100814,72771,68,390,1500,6576,24.375000,212.500000,6.533333,4.250000,7.0,0.000000,0.062500,0.687500,-1.812500,0.125000,0.437500,5.968601,0.500000,1.000000,0.937500,0.062500,0.000000,0.0,0.0000,0.0,0.0,0.0,16,0,0,2022-09-19 03:28:44,900,0,10,236.133634,0,0,1,0,0,0,1,0,0,1,0,0,1,0
100817,72772,188,388,1364,132,24.250000,191.562500,6.600000,3.562500,8.0,0.062500,0.062500,-1.875000,-0.125000,0.062500,0.375000,7.173735,0.625000,0.625000,0.937500,0.062500,0.000000,0.0,0.0000,0.0,0.0,0.0,16,1,0,2022-08-31 15:57:38,900,0,1,681.370077,0,0,1,0,0,0,1,0,0,0,0,1,0,1


# Submitting

In [31]:
submit_df = test_set[['game_id','rating']]
submit_df
submit_df.to_csv('submission.csv', index=False)